%md
#### Data Split
Reads the landing file and applies the Split rules, output as parquet format.

In [0]:
%run "/Shared/MDMF/Tools/utilities"

In [0]:
# Obtain the parameters sent by Azure Data Factory
#dbutils.widgets.removeAll()

# Input Parameters
dbutils.widgets.text("SourceInstanceURL", "", "")
dbutils.widgets.text("SourceContainerName", "", "")
dbutils.widgets.text("SourceFilePath", "", "")
dbutils.widgets.text("SourceFileExtension", "", "")
dbutils.widgets.text("ObjectName", "", "")
dbutils.widgets.text("ObjectSchema", "", "")
dbutils.widgets.text("ColumnSchema", "", "")
dbutils.widgets.text("InputParameters", "", "")
dbutils.widgets.text("SourceKeySecretName", "", "")
dbutils.widgets.text("NbRunParameters", "", "")

# Output Parameters
dbutils.widgets.text("OutputParameters", "", "")

In [0]:
##source variables
nb_run_parameters = json.loads(dbutils.widgets.get("NbRunParameters"))
source_instance_url = dbutils.widgets.get("SourceInstanceURL")
source_container_name = dbutils.widgets.get("SourceContainerName")
source_object_schema = json.loads(dbutils.widgets.get("ObjectSchema"))
source_column_schema = json.loads(dbutils.widgets.get("ColumnSchema"))
source_file_path = dbutils.widgets.get("SourceFilePath")
source_file_extension = dbutils.widgets.get("SourceFileExtension")
object_name = dbutils.widgets.get("ObjectName")
input_parameters = json.loads(dbutils.widgets.get("InputParameters"))
source_key_secret_name = dbutils.widgets.get("SourceKeySecretName")

##output variables
output_parameters = json.loads(dbutils.widgets.get("OutputParameters"))

#derived variables
kv_scope_name = nb_run_parameters['kvScopeName']
service_principal_id = nb_run_parameters['servicePrincipalId']
service_principal_secret_name = nb_run_parameters['servicePrincipalSecretName']
tenant_id = nb_run_parameters['tenantId']
storage_name = return_storage_name(source_instance_url)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-559456846063534> in <module> 
 1 ##source variables 
 ----> 2 nb_run_parameters = json . loads ( dbutils . widgets . get ( "NbRunParameters" ) ) 
 3 source_instance_url = dbutils . widgets . get ( "SourceInstanceURL" ) 
 4 source_container_name = dbutils . widgets . get ( "SourceContainerName" ) 
 5 source_object_schema = json . loads ( dbutils . widgets . get ( "ObjectSchema" ) ) 

 NameError : name 'json' is not defined

In [0]:
def split_module(df, dataset_type_dict):
  df.createOrReplaceTempView(object_name)
  df = spark.sql(f"""{dataset_type_dict["query"]}""")
  return df

def save_to_sink(df, dataset_type_dict):
  for each_output in dataset_type_dict["outputs"]:
    output_storage_name = return_storage_name(each_output["outputInstanceURL"])
    write_to_adls(kv_scope_name, service_principal_id, service_principal_secret_name, tenant_id, output_storage_name, each_output["outputContainerName"], each_output["outputFilePath"], each_output["outputFileExtension"], df, object_name = object_name, object_schema = source_object_schema)
  
  return None

In [0]:
try:
  ##read from ADLS (without schema)
  df = read_from_adls(kv_scope_name, service_principal_id, service_principal_secret_name, tenant_id, storage_name, source_container_name, source_file_path, source_file_extension, object_schema = source_object_schema)
  
  ##creates dataframes using input parameters and writes into ADLS
  for each_dataset_type in input_parameters:
    print("Reading from Input: ", input_parameters[each_dataset_type])
    df_split = split_module(df, input_parameters[each_dataset_type])
    print("Reading to Output: ", str(output_parameters[each_dataset_type]))
    save_to_sink(df_split, output_parameters[each_dataset_type])

except Exception as ex:
  raise Exception(f"Error: {ex}")

Reading from Input: {'query': 'SELECT C.* FROM Contract C'}
Reading to Output: {'outputs': [{'fwkTargetId': '2', 'outputInstanceURL': 'https://cldtppshcpdev0adlsstg.dfs.core.windows.net/', 'outputContainerName': 'landing', 'outputFilePath': 'DB2/MDM/mdmadms/Contract/Full/Contract_20220810203659', 'outputFileExtension': 'parquet', 'outputKeySecretName': ''}]}

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-559456846063536> in <module> 
 11 print ( "Reading to Output: " , str ( output_parameters [ each_dataset_type ] ) ) 
 ---> 12 save_to_sink ( df_split , output_parameters [ each_dataset_type ] ) 
 13 

 <command-559456846063535> in save_to_sink (df, dataset_type_dict) 
 8 output_storage_name = return_storage_name ( each_output [ "outputInstanceURL" ] ) 
 ----> 9 write_to_adls ( kv_scope_name , service_principal_id , service_principal_secret_name , tenant_id , output_storage_name , each_output [ "outputContainerName" ] , each_output [ "outputFilePath" ] , each_output [ "outputFileExtension" ] , df , object_name = object_name , object_schema = source_object_schema ) 
 10 

 <command-559456846063565> in write_to_adls (kv_scope_name, service_principal_id, service_principal_secret_name, tenant_id, storage_name, container_name, file_path, file_extension, df, object_schema, write_mode, object_name) 
 31 if file_extension == 'parquet' : 
 ---> 32 df . write . format ( "parquet" ) . mode ( write_mode ) . save ( full_file_path ) 
 33 elif file_extension == 'delta' : 

 /databricks/spark/python/pyspark/sql/readwriter.py in save (self, path, format, mode, partitionBy, **options) 
 739 else : 
 --> 740 self . _jwrite . save ( path ) 
 741 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 

 Py4JJavaError : An error occurred while calling o1518.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:606)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:360)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:198)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:126)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:124)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:138)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$$nestedInanonfun$eagerlyExecuteCommands$1$1.$anonfun$applyOrElse$1(QueryExecution.scala:160)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withCustomExecutionEnv$8(SQLExecution.scala:239)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:386)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withCustomExecutionEnv$1(SQLExecution.scala:186)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:968)
	at org.apache.spark.sql.execution.SQLExecution$.withCustomExecutionEnv(SQLExecution.scala:141)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:336)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$$nestedInanonfun$eagerlyExecuteCommands$1$1.applyOrElse(QueryExecution.scala:160)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$$nestedInanonfun$eagerlyExecuteCommands$1$1.applyOrElse(QueryExecution.scala:156)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$transformDownWithPruning$1(TreeNode.scala:575)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin

In [0]:
dbutils.notebook.exit("DSplit_01_DataSplit Notebook succeeded")